In [4]:
import requests
import pandas as pd
from datetime import date, datetime, timedelta
from numpy import sum

In [5]:
response = requests.get("https://iss.moex.com/iss/engines/stock/markets/bonds/boards/TQCB/securities.xml?iss.meta=off&iss.only=securities&securities.columns=SECID,SECNAME,SHORTNAME,COUPONVALUE,COUPONPERCENT,MATDATE,NEXTCOUPON,STATUS,FACEVALUE")

In [6]:
bonds = pd.read_xml(response.content, xpath=".//row")

In [7]:
bonds

,SECID,SECNAME,SHORTNAME,COUPONVALUE,COUPONPERCENT,MATDATE,NEXTCOUPON,STATUS,FACEVALUE
0,AMUNIBB2DER6,UNIBANK OJSC Series 1 21,UBANK02/24,1.33,5.300,2024-02-05,2023-05-05,A,100.0
1,RU000A0JQ7Z2,"""Российские ЖД"" ОАО 19 обл.",РЖД-19 обл,39.14,7.850,2024-07-08,2023-07-10,A,1000.0
2,RU000A0JQAL8,ДОМ.РФ (АО) обл. сер. А14,ДОМ.РФ14об,49.59,10.000,2023-05-15,2023-05-15,A,1000.0
3,RU000A0JQAM6,ДОМ.РФ (АО) обл. сер. А15,ДОМ.РФ15об,52.07,10.500,2028-09-15,2023-03-15,A,1000.0
4,RU000A0JQRD9,"""Российские ЖД"" ОАО 23 обл.",РЖД-23 обл,39.14,7.850,2025-01-16,2023-07-20,A,1000.0
...,...,...,...,...,...,...,...,...,...
2017,XS2392969395,CBOM Finance P.L.C. 7.625 UNDT,CBOM Finan,19.06,7.625,0000-00-00,2023-04-04,A,1000.0
2018,XS2393688598,SovCom Capital UNTD,SCB-UNTD,19.00,7.600,0000-00-00,2023-02-17,A,1000.0
2019,XS2415466676,BCS SP Plc Series 229,BCS11/24-8,10.00,0.010,2024-11-29,2023-11-29,A,100000.0
2020,XS2417535650,BCS SP Plc Series 234,BCS06/25-A,0.00,NaN,2025-06-01,2023-12-01,A,100000.0


In [8]:
bonds = bonds.dropna()
bonds = bonds.drop(
    bonds[
        (bonds.COUPONPERCENT < 9)
        | (bonds.COUPONPERCENT > 20)
        | (bonds.COUPONVALUE == 0)
        | (bonds.MATDATE == "0000-00-00")
        | (bonds.NEXTCOUPON == "0000-00-00")
        | (bonds.STATUS != "A")
        | (bonds.FACEVALUE != 1000)
    ].index
)


In [9]:
def delta(d):
    d = datetime.strptime(d["MATDATE"], '%Y-%m-%d').date()
    delta = d - date.today()
    y = delta.days // 365
    m = delta.days % 365 // 30
    if y > 0:
        return f"{y} years {m} months"
    return f"{m} months"

In [10]:
bonds["DELTA"] = bonds.apply(delta, axis=1)

In [11]:
bonds.sort_values("COUPONPERCENT").to_csv("1.csv")

In [15]:
def get_trades_number(row: str):
    date = datetime.today() - timedelta(7)
    date = date.strftime("%Y-%m-%d")
    response = requests.get(
        "https://iss.moex.com/iss/history/engines/stock/markets/bonds/boards/TQCB/securities/"
        + row["SECID"]
        + ".json?iss.meta=off&iss.only=history&history.columns=NUMTRADES&limit=20&from="
        + date
    )
    res = sum(response.json()["history"]["data"])
    print(res)
    return res

In [16]:
bonds["TRADES"] = bonds.apply(get_trades_number, axis=1)

60
23
0
1
11
0
174
17
46
0
0
0
0
0
0
22
0
0
0
0
0
192
0
0
72
0
0
40
0
0
0
0
0
94
6
0
0
0
0
0
0
0
0
10
0
1
0
0
1
2
194
0
8
5
10
45
282
2
0
0
0
0
0
101
0
0
0
0
11
0
0
50
8
0
17
7
58
0
0
110
47
0
0
0
0
0
0
0
0
0
0
0
159
0
250
682
166
304
484
0
47
13
0
0
46
291
0
0
2
103
0
0
76
50
0
1488
450
142
173
68
0
0
27
22
11
112
77
58
177
0
0
0
10
86
0
0
414
0
198
0
0
0
0
69
27
0
318
0
0
268
673
17
2
0
574
166
527
1018
7
201
79
45
738
375
1
565
725
497
175
66
47
200
21
6
0
180
118
37
120
11
1472
0
0
0
7
9
808
17
79
0
28
417
917
130
114
0
84
75
2750
490
119
0
0
471
474
332
286
264
22
619
686
72
373
2463
151
0
1294
248
316
637
70
494
0
259
2270
784
520
790
1062
229
152
306
79
1515
1223
841
226
605
599
53
69
480
355
227
201
123
283
21
221
348
554
284
253
1307
100
2559
667
682
796
103
464
437
319
1068
88
87
1119
132
546
310
0
295
0
225
169
1045
102
245
91
235
0
57
166
168
647
0
106
0
380
16
0
45
105
528
78
242
1622
1861
141
1586
31
134
436
236
301
105
140
263
0
172
368
20
0
673
0
61
82
237
116
17
160
0


In [17]:
bonds = bonds.drop(bonds[(bonds.TRADES < 10)].index)

In [20]:
bonds = bonds.drop(bonds[(bonds.COUPONPERCENT < 10)].index)

In [21]:
bonds.to_csv("1.csv")